In [2]:
import numpy  as np
import pandas as pd

# visualization libraries
import matplotlib.pyplot as plt

import cv2
import torch
import torch.nn.functional as F
import uuid

# Live WebCam Demo Inception-V3

In [3]:
%pwd

'c:\\Users\\kings\\OneDrive - MMU\\MSC DATA SCIENCE\\facial_expression\\notebooks'

In [4]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 3)
state_dict = torch.load("../models/Inception3_Adamproposed_augmediapipe.pt")

model.load_state_dict(state_dict)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

Using cache found in C:\Users\kings/.cache\torch\hub\pytorch_vision_v0.10.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [5]:
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

In [ ]:
# def predict_emotion(output):
#     font_scale = 1.5
#     font = cv2.FONT_HERSHEY_PLAIN
#     cap = cv2.VideoCapture(output)
#     if not cap.isOpened():
#         raise IOError("Cannot open webcam")
#     while True:
#         # Capture a frame
#         ret, frame = cap.read()
#         facedet = cv2.CascadeClassifier(r'C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\Msc-Project\haarcascades\haarcascade_frontalface_default.xml')
#         gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         faces = facedet.detectMultiScale(gray_frame, 1.1, 4)

#         if len(faces) == 0:
#             print("No faces detected in the frame")
#             continue  # Skip processing this frame if no faces are detected

#         for x, y, w, h in faces:
#             roi_gray = gray_frame[y:y+h, x:x+w]
#             roi_color = frame[y:y+h, x:x+w]
#             cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
#             facess = facedet.detectMultiScale(roi_gray)
#             if len(facess) == 0:
#                 print("Face not detected")
#             else:
#                 for (ex, ey, ew, eh) in facess:
#                     face_roi = roi_color[ey:ey+eh, ex:ex+ew]  # cropping the face

#         rgb_image = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
#         image = cv2.resize(rgb_image, (299, 299))
#         image = image / 255.0  # normalization
#         data = torch.from_numpy(image)
#         data = data.type(torch.FloatTensor)
#         data = data.to(device)
#         reshaped_data = data.permute(2, 0, 1)  # Reshape the tensor to have 3 as the first dimension
#         reshaped_data = reshaped_data.unsqueeze(0)  # add a fourth dimension
#         outputs = model(reshaped_data)
#         pred = F.softmax(outputs[0], dim=-1)
#         final_pred = torch.argmax(pred, 0)

#         if final_pred == 0:
#             emotion = "Negative"
#         elif final_pred == 1:
#             emotion = "Neutral"
#         else:
#             emotion = "Positive"

#         text_x, text_y = x, y - 10  # Adjust the position above the bounding box
#         cv2.putText(frame, emotion, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
#         cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

#         conf, classes = torch.max(pred, -1)
#         emotion_id = [0, 1, 2]
#         class_dict = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
#         class_names = [class_dict[label] for label in emotion_id]
#         result_data = []
#         result_data.append([conf.item(), pred.tolist(), class_names[classes.item()]])

#         # Write the result to a CSV file
#         with open(output+'.csv', mode='a', newline='') as file:
#             writer = csv.writer(file)
#             if file.tell() == 0:
#                 writer.writerow(['Confidence', 'max_confidence', 'Emotion'])  # Write header only if the file is empty
#             writer.writerows(result_data)

#         cv2.imshow("Facial Expression Recognition", frame)

#         if cv2.waitKey(2) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()


In [ ]:
import cv2
import csv
import torch
import torch.nn.functional as F
import numpy as np
from pathlib import Path

def create_prediction_video(frames, storage, timestamp, model_type, fps=4, is_color=True):
    """Creates a video from a list of frames and saves it."""
    video_path = storage / f"pred_{model_type}_{timestamp}.mp4"
    height, width = frames[0].shape[:2]

    # Ensure frames are in uint8 format
    frames_uint8 = [(frame * 255).astype(np.uint8) if frame.max() <= 1 else frame.astype(np.uint8) for frame in frames]

    # Initialize VideoWriter
    out = cv2.VideoWriter(str(video_path),
                          cv2.VideoWriter_fourcc(*'avc1'),
                          fps, (width, height),
                          is_color)

    if not out.isOpened():
        out = cv2.VideoWriter(str(video_path),
                              cv2.VideoWriter_fourcc(*'MJPG'),
                              fps, (width, height),
                              is_color)

    for frame in frames_uint8:
        out.write(frame)

    out.release()
    print(f"Saved video at: {video_path}")
    return str(video_path)


def predict_emotion(video_path, output_folder):
    """Processes a video, detects emotions, and saves results + processed video."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Cannot open video")
    
    facedet = cv2.CascadeClassifier(r'C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\MSC Project\Msc-Project\haarcascades\haarcascade_frontalface_default.xml')
    
    processed_frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Stop if video ends

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facedet.detectMultiScale(gray_frame, 1.1, 4)

        if len(faces) == 0:
            print("No faces detected in the frame")
            continue

        for x, y, w, h in faces:
            roi_gray = gray_frame[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        rgb_image = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2RGB)
        image = cv2.resize(rgb_image, (299, 299))
        image = image / 255.0  # nomalization
        data = torch.from_numpy(image).type(torch.FloatTensor).to(device)
        reshaped_data = data.permute(2, 0, 1).unsqueeze(0)  # Reshape the tensor to have 3 as the first dimension and adjust dimensions for model

        outputs = model(reshaped_data)
        pred = F.softmax(outputs[0], dim=-1)
        final_pred = torch.argmax(pred, 0)

        emotion = ["Negative", "Neutral", "Positive"][final_pred.item()]
        text_x, text_y = x, y - 10
        cv2.putText(frame, emotion, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

        # Confidence and emotion label
        conf, classes = torch.max(pred, -1)
        result_data = [[conf.item(), pred.tolist(), emotion]]

        # Save results to CSV
        csv_path = Path(output_folder) / (Path(video_path).stem + '_results.csv')
        with open(csv_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0:
                writer.writerow(['Confidence', 'max_confidence', 'Emotion'])
            writer.writerows(result_data)

        cv2.imshow("Facial Expression Recognition", frame)
        processed_frames.append(frame)

        if cv2.waitKey(2) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Save processed frames as a video
    timestamp = Path(video_path).stem
    storage_path = Path(output_folder)
    saved_video_path = create_prediction_video(processed_frames, storage_path, timestamp, "emotion_detection", fps=4, is_color=True)

    print(f"Video saved at: {saved_video_path}")


In [7]:
import os

In [9]:
preds = predict_emotion(r"C:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\facial_expression\static\videos\video_test.mp4", os.getcwd())

Saved video at: c:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\facial_expression\notebooks\pred_emotion_detection_video_test.mp4
Video saved at: c:\Users\kings\OneDrive - MMU\MSC DATA SCIENCE\facial_expression\notebooks\pred_emotion_detection_video_test.mp4
